In [1]:
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt

In [111]:
db = pd.read_csv("Sample-Superstore.csv",encoding='latin1' )

In [120]:
rename_dict = {
        'Row ID': 'RowID',
        'Order ID': 'OrderID',
        'Order Date': 'OrderDate',
        'Ship Date': 'ShipDate',
        'Ship Mode': 'ShipMode',
        'Customer ID': 'CustomerID',
        'Customer Name': 'CustomerName',
        'Postal Code': 'PostalCode',
        'Product ID': 'ProductID',
        'Sub-Category': 'SubCategory',
        'Product Name': 'ProductName',
        'Catalogue price': 'CataloguePrice',
        'Supplier price': 'SupplierPrice'
        }

# call rename () method
db.rename(columns=rename_dict,
          inplace=True)

In [133]:
db[['OrderDate','ShipDate']] = db[['OrderDate','ShipDate']].apply(pd.to_datetime)
db.head()

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,Price,DiscountedPrice,PurchasePrice
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,130.98,130.9800,110.0232
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,243.98,243.9800,170.7860
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,7.31,7.3100,3.8743
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,348.21,191.5155,268.1217
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,13.98,11.1840,9.9258


In [132]:
db["Price"] = (db.Sales/db.Quantity)/(1-db.Discount)


db["DiscountedPrice"] = (db.Sales/db.Quantity)
db["PurchasePrice"] = (db.Sales-db.Profit)/db.Quantity

In [159]:
#Create rables dataset
Customers = db[["CustomerID", "CustomerName", "Segment"]]
Address = db[["CustomerID","Country" ,"Region", "State", "City", "PostalCode"]] # Address ID will be required to connect this to orders. Removed country as not required
Products = db[["ProductID", "ProductName","Category","SubCategory"]] 
ProductPrice = db[["ProductID","PurchasePrice", "Price", "DiscountedPrice"]]
Orders = db[["OrderID", "CustomerID", "OrderDate", "ShipDate", "ShipMode"]] #Needs Address ID!
Sales = db[["OrderID", "ProductID", "Quantity", "Sales", "Discount", "Profit"]] #Sales table did not survive 3NF

In [324]:
Address = db[["CustomerID","Country" ,"Region", "State", "City", "PostalCode"]]

In [160]:
Customers["CustomerID"].unique().size
Customers["CustomerName"].unique().size
Customers = Customers.drop_duplicates(ignore_index=True)
Customers

,CustomerID,CustomerName,Segment
0,CG-12520,Claire Gute,Consumer
1,DV-13045,Darrin Van Huff,Corporate
2,SO-20335,Sean O'Donnell,Consumer
3,BH-11710,Brosina Hoffman,Consumer
4,AA-10480,Andrew Allen,Consumer
...,...,...,...
788,CJ-11875,Carl Jackson,Corporate
789,RS-19870,Roy Skaria,Home Office
790,SC-20845,Sung Chung,Consumer
791,RE-19405,Ricardo Emerson,Consumer


In [144]:
Address.isnull().sum()

CustomerID    0
Region        0
State         0
City          0
PostalCode    0
dtype: int64

In [329]:
#create AdressID
#Address.CustomerID.nunique()
Address.groupby(by=["CustomerID", "PostalCode"]).ngroups
Address["PostID"]=Address.groupby(['CustomerID','Country', 'City', 'State', 'PostalCode', 'Region']).ngroup() +1
Address["PostID"] = "A" + Address['PostID'].astype(str).str.zfill(5)
Address=Address.drop_duplicates()
Address=Address.reset_index(drop=True)
#db["PostID"] = Address.PostID


/var/folders/_f/kkvs81ts6bn_m7y2gqnp_xjm0000gn/T/ipykernel_74240/196637436.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Address["PostID"]=Address.groupby(['CustomerID','Country', 'City', 'State', 'PostalCode', 'Region']).ngroup() +1
/var/folders/_f/kkvs81ts6bn_m7y2gqnp_xjm0000gn/T/ipykernel_74240/196637436.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Address["PostID"] = "A" + Address['PostID'].astype(str).str.zfill(5)


In [196]:
Products = Products.drop_duplicates(subset=["ProductID", "ProductName"])

In [197]:
duplicated_products=Products.groupby("ProductID")["ProductName"].nunique()
duplicated_products = duplicated_products[duplicated_products > 1].index
Products['NewProductID'] = Products['ProductID']
# Iterate through the duplicated ProductIDs and assign new IDs for each unique ProductName
for product_id in duplicated_products:
    # Filter rows with this ProductID
    product_group = Products[Products['ProductID'] == product_id]
    
    # Enumerate through the rows with this ProductID and assign new unique ProductIDs
    for i, idx in enumerate(product_group.index):
        # Create a new ProductID by appending a suffix or a numeric sequence
        Products.at[idx, 'NewProductID'] = f"{product_id}-{i+1}"

# Step 3: Inspect the result
# Display the DataFrame with old and new ProductIDs
Products[['ProductID', 'ProductName', 'NewProductID']].sort_values(by="ProductID",ascending=True).head(30)




,ProductID,ProductName,NewProductID
3512,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",FUR-BO-10000112
1760,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",FUR-BO-10000330
1594,FUR-BO-10000362,Sauder Inglewood Library Bookcases,FUR-BO-10000362
1932,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,FUR-BO-10000468
4088,FUR-BO-10000711,"Hon Metal Bookcases, Gray",FUR-BO-10000711
2403,FUR-BO-10000780,O'Sullivan Plantations 2-Door Library in Landv...,FUR-BO-10000780
485,FUR-BO-10001337,O'Sullivan Living Dimensions 2-Shelf Bookcases,FUR-BO-10001337
841,FUR-BO-10001519,O'Sullivan 3-Shelf Heavy-Duty Bookcases,FUR-BO-10001519
7967,FUR-BO-10001567,"Bush Westfield Collection Bookcases, Dark Cher...",FUR-BO-10001567
472,FUR-BO-10001601,"Sauder Mission Library with Doors, Fruitwood F...",FUR-BO-10001601


In [209]:
#Products.drop("ProductID", axis=1, inplace=True)
Products.rename(columns={"NewProductID":"ProductID"}, inplace=True)
Products.index


Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       9214, 9319, 9378, 9455, 9456, 9521, 9562, 9604, 9673, 9935],
      dtype='int64', length=1894)

In [278]:
#Change also the ProductID of other tables 
#ProductPrice[ProductPrice["index"].isin(Products.index)]["ProductID"]= Products.ProductID
#Sales.iloc[Products.index,[0]] = Products["ProductID"]
#ProductPrice=ProductPrice.reset_index()
#Products.reset_index(inplace=True)

In [253]:
#ProductPrice.drop_duplicates(inplace=True)
ProductPrice["PriceID"]=ProductPrice.groupby(['ProductID', 'PurchasePrice', 'Price', 'DiscountedPrice']).ngroup() + 1
ProductPrice["PriceID"] = "P" + ProductPrice["PriceID"].astype(str).str.zfill(5)
#["PriceID"]
ProductPrice.reset_index(inplace=True)



/var/folders/_f/kkvs81ts6bn_m7y2gqnp_xjm0000gn/T/ipykernel_74240/1476352983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ProductPrice["PriceID"]=ProductPrice.groupby(['ProductID', 'PurchasePrice', 'Price', 'DiscountedPrice']).ngroup() + 1
/var/folders/_f/kkvs81ts6bn_m7y2gqnp_xjm0000gn/T/ipykernel_74240/1476352983.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ProductPrice["PriceID"] = "P" + ProductPrice["PriceID"].astype(str).str.zfill(5)


In [321]:
#Remove dublicate from Orders

#Orders = Orders.drop_duplicates()
Orders

,OrderID,CustomerID,OrderDate,ShipDate,ShipMode
0,CA-2016-152156,CG-12520,2016-11-08,2016-11-11,Second Class
2,CA-2016-138688,DV-13045,2016-06-12,2016-06-16,Second Class
3,US-2015-108966,SO-20335,2015-10-11,2015-10-18,Standard Class
5,CA-2014-115812,BH-11710,2014-06-09,2014-06-14,Standard Class
12,CA-2017-114412,AA-10480,2017-04-15,2017-04-20,Standard Class
...,...,...,...,...,...
9986,CA-2016-125794,ML-17410,2016-09-29,2016-10-03,Standard Class
9987,CA-2017-163629,RA-19885,2017-11-17,2017-11-21,Standard Class
9989,CA-2014-110422,TB-21400,2014-01-21,2014-01-23,Second Class
9990,CA-2017-121258,DB-13060,2017-02-26,2017-03-03,Standard Class


In [327]:
Shipin_details =db[["CustomerID","ShipDate", "ShipMode", "PostID"]].drop_duplicates()
Shipin_details

,CustomerID,ShipDate,ShipMode,PostID
0,CG-12520,2016-11-11,Second Class,A00860
2,DV-13045,2016-06-16,Second Class,A01437
3,SO-20335,2015-10-18,Standard Class,A04367
5,BH-11710,2014-06-14,Standard Class,A00539
12,AA-10480,2017-04-20,Standard Class,A00015
...,...,...,...,...
9986,ML-17410,2016-10-03,Standard Class,A03206
9987,RA-19885,2017-11-21,Standard Class,A03806
9989,TB-21400,2014-01-23,Second Class,A04568
9990,DB-13060,2017-03-03,Standard Class,A01134


In [306]:
new_db = pd.merge( Address,Shipin_details, on='CustomerID',)

# Display the merged DataFrame
new_db[["OrderID","ShipDate", "ShipMode", "PostID"]].drop_duplicates()

,OrderID,ShipDate,ShipMode,PostID
0,CA-2016-152156,2016-11-11,Second Class,A00860
1,CA-2017-164098,2017-01-27,First Class,A00860
2,US-2015-123918,2015-10-15,Same Day,A00860
3,CA-2016-138688,2016-06-16,Second Class,A01437
4,CA-2017-139773,2017-12-04,Same Day,A01437
...,...,...,...,...
35980,US-2017-107888,2017-11-19,First Class,A00792
35981,CA-2016-131380,2016-03-31,Second Class,A00792
35982,CA-2014-161634,2014-11-20,Standard Class,A00792
35983,CA-2014-103527,2014-09-14,Second Class,A00792


In [339]:
Customers
Address
Shipin_details
Orders
ProductPrice
Sales

,OrderID,ProductID,Quantity,Sales,Discount,Profit
0,FUR-BO-10001798,FUR-BO-10001798,2,261.9600,0.00,41.9136
1,FUR-CH-10000454,FUR-CH-10000454,3,731.9400,0.00,219.5820
2,OFF-LA-10000240,OFF-LA-10000240,2,14.6200,0.00,6.8714
3,FUR-TA-10000577,FUR-TA-10000577,5,957.5775,0.45,-383.0310
4,OFF-ST-10000760,OFF-ST-10000760,2,22.3680,0.20,2.5164
...,...,...,...,...,...,...
9989,CA-2014-110422,FUR-FU-10001889,3,25.2480,0.20,4.1028
9990,CA-2017-121258,FUR-FU-10000747,2,91.9600,0.00,15.6332
9991,CA-2017-121258,TEC-PH-10003645,2,258.5760,0.20,19.3932
9992,CA-2017-121258,OFF-PA-10004041,4,29.6000,0.00,13.3200
